# EDA
> Exploratory analysis of the results

In [2]:
import pandas as pd
from fastcore.xtras import Path

In [26]:
report_path = Path("../scripts/reports/nlp_script_accelerate/")

In [27]:
initial = report_path.ls()[0]; initial

Path('../scripts/reports/nlp_script_accelerate/multi_gpu_adjusted_bs')

In [45]:
columns = ["accelerator", "num_processes", "iteration", "learning_rate", "train_batch_size", "validation_batch_size"]
for i in range(1,4):
    columns.append(f"epoch_{i}_accuracy")
    columns.append(f"epoch_{i}_f1")
    columns.append(f'epoch_{i}_train_time')
    columns.append(f'epoch_{i}_validation_time')
columns += ["total_time"]

In [55]:
df = pd.DataFrame(columns=columns)

In [56]:
def process_one(df, path):
    for i,file in enumerate(Path(path).ls()):
        if file.is_file():
            j = file.read_json()
            data = [j["training_device"], j["num_devices"], i, j["lr"], j["train_batch_size"], j["eval_batch_size"]]
            mets = j["metrics"]
            speeds = j["speeds"]
            for i in range(3):
                data += [mets[i]["accuracy"], mets[i]["f1"]]
                data += [speeds["training"]["times"][i], speeds["evaluation"]["times"][i]]
            data += [speeds["total"]]
            df.loc[len(df.index)] = data
    return df

In [57]:
for folder in Path("../scripts/reports/nlp_script_accelerate").ls():
    if not folder.is_file():
        df = process_one(df, folder)

In [59]:
df.to_csv("../scripts/reports/nlp_script_accelerate/results.csv")